In [1]:
import tensorflow as tf
import os
import zipfile

DS_URL = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip"
DS_LOCAL = "/tmp/happy-or-sad.zip"
DS_DIR = "/tmp/h-or-s"
DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    DS_URL \
    -O DS_LOCAL

zip_ref = zipfile.ZipFile(DS_LOCAL, 'r')
zip_ref.extractall(DS_DIR)
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        DS_DIR,  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=2,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])



--2019-02-18 18:01:10--  http://ds_url/
Resolving ds_url (ds_url)... failed: Name or service not known.
wget: unable to resolve host address ‘ds_url’
Instructions for updating:
Colocations handled automatically by placer.
Found 80 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
8/8 [==============================] - 3s 331ms/step - loss: 1.2184 - acc: 0.5000
Epoch 2/15
8/8 [==============================] - 3s 315ms/step - loss: 0.2908 - acc: 0.9000
Epoch 3/15
8/8 [==============================] - 2s 310ms/step - loss: 0.4233 - acc: 0.8625
Epoch 4/15
8/8 [==============================] - 3s 316ms/step - loss: 0.2051 - acc: 0.9125
Epoch 5/15
8/8 [==============================] - 2s 312ms/step - loss: 0.1253 - acc: 0.9625
Epoch 6/15
8/8 [==============================] - 3s 316ms/step - loss: 0.0659 - acc: 0.9750
Epoch 7/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0175 - acc: 1.0000
Reached 99.9% accuracy so cancelling training

In [0]:
""